# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[2]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names 
flipkart_df.select('maincateg').distinct().show()  

+---------+
|maincateg|
+---------+
|     null|
|      Men|
|    Women|
+---------+



In [0]:
# Filter products with rating > 4 and more than 100 reviews
flipkart_df.select("id","title","rating","noreviews1").where((flipkart_df["Rating"] > 4) & (flipkart_df["noreviews1"] >100)).show()

+-----+--------------------+------+----------+
|   id|               title|rating|noreviews1|
+-----+--------------------+------+----------+
|13859|Women Black Wedge...|   4.2|       487|
|19453|Denill Ankle Leng...|   4.1|       961|
| 3205|     SFG-23 Slippers|   4.1|      1598|
| 7625|White casual, gym...|   4.2|       381|
|17791|Synthetic Leather...|   4.2|       182|
|15762|Running Shoes For...|   4.1|      8254|
|15247|SM-484 Casuals Fo...|   4.2|       436|
|20247|Women Black Heels...|   4.1|       182|
|10655|Women Brown Wedge...|   4.1|       178|
|20918|GO WALK 5-SPARROW...|   4.3|       106|
|16193|crocs Unisex's Ra...|   4.3|       661|
| 8485|Men Grey Clogs Sa...|   4.1|       328|
| 7561|Men Navy Sports S...|   4.2|      2103|
| 9249|Women Black Wedge...|   4.2|       487|
|20597|Sneakers For Men ...|   4.1|       597|
|11078|Men Black Clogs S...|   4.2|      1098|
|13151|Walking Shoes For...|   4.1|       938|
|15716|Mojaris For Men  ...|   4.1|       106|
|20324|Women 

In [0]:
# Display Products in 'Men' category that are fulfilled
flipkart_df.filter((flipkart_df["maincateg"] == "Men") & (flipkart_df["fulfilled1"] == 1)).select("title", "maincateg", "fulfilled1").show()

+--------------------+---------+----------+
|               title|maincateg|fulfilled1|
+--------------------+---------+----------+
|Casuals For Men  ...|      Men|         1|
|Men's Height Incr...|      Men|         1|
|Combo Pack Of 2 C...|      Men|         1|
|MOCCASSIN Slip On...|      Men|         1|
|Long OfficeWear C...|      Men|         1|
|     SFG-23 Slippers|      Men|         1|
|Dark Brown Leathe...|      Men|         1|
|Long OfficeWear C...|      Men|         1|
|Monk Strap For Me...|      Men|         1|
|    Men Brown Sandal|      Men|         1|
|Men Hawaii Flip F...|      Men|         1|
|Men Blue Clogs Sa...|      Men|         1|
|Java 2 IDP Flip F...|      Men|         1|
|Derby For Men  (B...|      Men|         1|
|Combo pack of 2 s...|      Men|         1|
|GO WALK 5-SPARROW...|      Men|         1|
|Genuine Leather F...|      Men|         1|
|Genuine Leather F...|      Men|         1|
|Men Navy Sports S...|      Men|         1|
|    Men Brown Sandal|      Men|

In [0]:
# Display number of products per category
flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category
flipkart_df.groupBy("maincateg").agg({"Rating": "avg"}).show()

+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews
flipkart_df.groupBy("maincateg").agg({"noreviews1": "avg"}).orderBy("avg(noreviews1)", ascending=False).show(1)


+---------+-----------------+
|maincateg|  avg(noreviews1)|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price
flipkart_df.select("title", "actprice1").orderBy("actprice1", ascending=False).limit(5).show()

+--------------------+---------+
|               title|actprice1|
+--------------------+---------+
|GEL-QUANTUM 360 K...|    15999|
|Women Orange Spor...|    12999|
|GT-2000 5 (D) Run...|    10999|
|Hybrid Astro Runn...|    10999|
|DEERUPT RUNNER W ...|     9999|
+--------------------+---------+



In [0]:
# Display Min, max, and avg price per category
flipkart_df.groupBy("maincateg").agg(
    {"actprice1": "min", "actprice1": "max", "actprice1": "avg"}
).show()

+---------+------------------+
|maincateg|    avg(actprice1)|
+---------+------------------+
|     null|1574.7313432835822|
|      Men|  1455.86013986014|
|    Women|1319.7081134564644|
+---------+------------------+



In [0]:
# Display number of nulls in each column
from pyspark.sql.functions import col, sum
flipkart_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in flipkart_df.columns]).show()

+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products
flipkart_df.groupBy("maincateg", "fulfilled1").count().show()

+---------+----------+-----+
|maincateg|fulfilled1|count|
+---------+----------+-----+
|      Men|         1| 1352|
|     null|         1|   40|
|    Women|         0| 1254|
|      Men|         0|  793|
|    Women|         1| 1778|
|     null|         0|   27|
+---------+----------+-----+



In [0]:
# Display Count of products per category
flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category
flipkart_df.groupBy("maincateg").agg({"Rating": "avg"}).show()

+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews
flipkart_df.groupBy("maincateg").agg({"noreviews1": "avg"}).orderBy("avg(noreviews1)", ascending=False).show(1)

+---------+-----------------+
|maincateg|  avg(noreviews1)|
+---------+-----------------+
|      Men|759.6554778554779|
+---------+-----------------+
only showing top 1 row



In [0]:
# Display Bar chart of product count per category
category_counts = flipkart_df.groupBy("maincateg").count()
# Use Databricks UI to visualize this as a bar chart
category_counts.display()


maincateg,count
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category
category_avg_rating = flipkart_df.groupBy("maincateg").agg({"Rating": "avg"})
category_avg_rating.display()

maincateg,avg(Rating)
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
category_reviews = flipkart_df.groupBy("maincateg").agg({"noreviews1": "sum"})
category_reviews.display()

maincateg,sum(noreviews1)
null,14084
Men,1629461
Women,535290


Databricks visualization. Run in Databricks to view.

In [0]:
# Display product name and 5 star rating for those products which have highest 5 star rating
flipkart_df.filter(flipkart_df["star_5f"] > 0).select("title", "star_5f").orderBy("star_5f", ascending=False).show()

+--------------------+-------+
|               title|star_5f|
+--------------------+-------+
|Jio-13 Running Sh...| 151193|
|Running Shoes For...| 151193|
|Running Shoes For...| 151173|
|Jio-13 Running Sh...| 151160|
|Jio-13 Running Sh...| 151159|
|Jio-13 Running Sh...| 101250|
|VERTIGO BLK RUNNI...|  66369|
|Cosko Sports Shoe...|  64987|
|Cosko Sports Shoe...|  64987|
|Cosko Sports Shoe...|  64981|
|Men Brown, Beige ...|  61722|
|    Men Olive Sandal|  61722|
|Oxygen Running Sh...|  55231|
|Boxer-303 Sneaker...|  54272|
|Boxer-303 Sneaker...|  54272|
|Mesh,Cricket, Bad...|  45128|
|Mesh,Cricket, Bad...|  45128|
|Mesh,Cricket, Bad...|  45128|
|Mesh,Cricket, Bad...|  45127|
|SOCKS Running Sho...|  43374|
+--------------------+-------+
only showing top 20 rows

